# Modules

In [2]:
import numpy as np
import pandas as pd

In [3]:
import os
import pprint
import tempfile

In [4]:
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs
from tensorflow.keras import layers

2024-09-09 17:48:07.871567: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-09 17:48:08.192339: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-09 17:48:08.192447: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-09 17:48:08.240145: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-09 17:48:08.255711: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-09 17:48:08.256858: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [7]:
from typing import Dict, Text

In [6]:
from datetime import datetime
import re

# Data

In [8]:
# Ratings data.
ratings = tfds.load("movielens/100k-ratings", split="train")
# Features of all the available movies.
movies = tfds.load("movielens/100k-movies", split="train")

2024-09-09 17:48:14.280176: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-09-09 17:48:14.280644: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [9]:
ratings_df = tfds.as_dataframe(ratings)
movies_df = tfds.as_dataframe(movies)

In [14]:
ratings_df.head(5)

,bucketized_user_age,movie_genres,movie_id,movie_title,raw_user_age,timestamp,user_gender,user_id,user_occupation_label,user_occupation_text,user_rating,user_zip_code
0,45.0,[7],b'357',"b""One Flew Over the Cuckoo's Nest (1975)""",46.0,879024327,True,b'138',4,b'doctor',4.0,b'53211'
1,25.0,"[4, 14]",b'709',b'Strictly Ballroom (1992)',32.0,875654590,True,b'92',5,b'entertainment',2.0,b'80525'
2,18.0,[4],b'412',"b'Very Brady Sequel, A (1996)'",24.0,882075110,True,b'301',17,b'student',4.0,b'55439'
3,50.0,"[5, 7]",b'56',b'Pulp Fiction (1994)',50.0,883326919,True,b'60',4,b'healthcare',4.0,b'06472'
4,50.0,"[10, 16]",b'895',b'Scream 2 (1997)',55.0,891409199,True,b'197',18,b'technician',3.0,b'75094'


In [16]:
user_id=str(933)

# Filter the dataset for user_id = '123'
filtered_ratings = ratings.filter(lambda x: x['user_id'] == tf.constant(user_id))

# Iterate through the filtered dataset to access the records
for record in filtered_ratings:
    movie_title = record['movie_title'].numpy().decode('utf-8')
    user_genre = record['movie_genres'].numpy()
    print(f"Movie: {movie_title}, Genre: {user_genre}")

Movie: Birdcage, The (1996), Genre: [4]
Movie: Braveheart (1995), Genre: [ 0  7 18]
Movie: Return of the Pink Panther, The (1974), Genre: [4]
Movie: Three Musketeers, The (1993), Genre: [0 1 4]
Movie: Romeo Is Bleeding (1993), Genre: [ 5 16]
Movie: Grease 2 (1982), Genre: [ 4 12 14]
Movie: Natural Born Killers (1994), Genre: [ 0 16]
Movie: Clockwork Orange, A (1971), Genre: [15]
Movie: City Slickers II: The Legend of Curly's Gold (1994), Genre: [ 4 19]
Movie: Star Wars (1977), Genre: [ 0  1 14 15 18]
Movie: Sting, The (1973), Genre: [4 5]
Movie: Fargo (1996), Genre: [ 5  7 16]
Movie: Star Trek: The Wrath of Khan (1982), Genre: [ 0  1 15]
Movie: Mulholland Falls (1996), Genre: [ 5  9 16]
Movie: Star Trek IV: The Voyage Home (1986), Genre: [ 0  1 15]
Movie: One Flew Over the Cuckoo's Nest (1975), Genre: [7]
Movie: Aliens (1986), Genre: [ 0 15 16 18]
Movie: Pulp Fiction (1994), Genre: [5 7]
Movie: Tombstone (1993), Genre: [19]
Movie: Silence of the Lambs, The (1991), Genre: [ 7 16]
Movie:

Movie: Blade Runner (1982), Genre: [ 9 15]
Movie: Rosencrantz and Guildenstern Are Dead (1990), Genre: [4 7]
Movie: In the Name of the Father (1993), Genre: [7]
Movie: Rock, The (1996), Genre: [ 0  1 16]
Movie: Man Without a Face, The (1993), Genre: [7]
Movie: Brazil (1985), Genre: [15]


In [8]:
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"],
    "user_ratings": float(x["user_rating"])
})
movies = movies.map(lambda x: x["movie_title"])

# Model

In [41]:
print('Total Data: {}'.format(len(ratings)))

tf.random.set_seed(42)

train = ratings.take(80_000)
test = ratings.skip(80_000).take(20_000)

Total Data: 100000


In [81]:
train

<_TakeDataset element_spec={'movie_title': TensorSpec(shape=(), dtype=tf.string, name=None), 'user_id': TensorSpec(shape=(), dtype=tf.string, name=None), 'user_ratings': TensorSpec(shape=(), dtype=tf.float32, name=None)}>

In [42]:
movie_titles = movies.batch(1_000)
user_ids = ratings.batch(1_000).map(lambda x: x["user_id"])

In [43]:
unique_movie_titles = np.unique(np.concatenate(list(movie_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

In [44]:
class MovieModel(tfrs.models.Model):

  def __init__(self, rating_weight: float, retrieval_weight: float) -> None:
    # We take the loss weights in the constructor: this allows us to instantiate
    # several model objects with different loss weights.

    super().__init__()

    embedding_dimension = 64

    # User and movie models.
    self.movie_model: tf.keras.layers.Layer = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_movie_titles, mask_token=None),
      tf.keras.layers.Embedding(len(unique_movie_titles) + 1, embedding_dimension)
    ])
    self.user_model: tf.keras.layers.Layer = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_user_ids, mask_token=None),
      tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
    ])

    # A small model to take in user and movie embeddings and predict ratings.
    # We can make this as complicated as we want as long as we output a scalar
    # as our prediction.
    self.rating_model = tf.keras.Sequential([
        tf.keras.layers.Dense(256, activation="relu"),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dense(1),
    ])

    # The tasks.
    self.rating_task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
        loss=tf.keras.losses.MeanSquaredError(),
        metrics=[tf.keras.metrics.RootMeanSquaredError()],
    )
    self.retrieval_task: tf.keras.layers.Layer = tfrs.tasks.Retrieval(
        metrics=tfrs.metrics.FactorizedTopK(
            candidates=movies.batch(128).map(self.movie_model)
        )
    )

    # The loss weights.
    self.rating_weight = rating_weight
    self.retrieval_weight = retrieval_weight

  def call(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    user_embeddings = self.user_model(features["user_id"])
    # And pick out the movie features and pass them into the movie model.
    movie_embeddings = self.movie_model(features["movie_title"])
    
    return (
        user_embeddings,
        movie_embeddings,
        # We apply the multi-layered rating model to a concatentation of
        # user and movie embeddings.
        self.rating_model(
            tf.concat([user_embeddings, movie_embeddings], axis=1)
        ),
    )

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:

    ratings = features.pop("user_ratings")

    user_embeddings, movie_embeddings, rating_predictions = self(features)

    # We compute the loss for each task.
    rating_loss = self.rating_task(
        labels=ratings,
        predictions=rating_predictions,
    )
    retrieval_loss = self.retrieval_task(user_embeddings, movie_embeddings)

    # And combine them using the loss weights.
    return (self.rating_weight * rating_loss
            + self.retrieval_weight * retrieval_loss)

In [45]:
model = MovieModel(rating_weight=1.0, retrieval_weight=1.0)

In [46]:
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

In [47]:
cached_train = train.batch(1_024).cache()
cached_test = test.batch(1_024).cache()

In [48]:
model.fit(cached_train, epochs=3)

Epoch 1/3
79/79 [==============================] - 7s 81ms/step - root_mean_squared_error: 1.2776 - factorized_top_k/top_1_categorical_accuracy: 1.8750e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0067 - factorized_top_k/top_10_categorical_accuracy: 0.0169 - factorized_top_k/top_50_categorical_accuracy: 0.1071 - factorized_top_k/top_100_categorical_accuracy: 0.2006 - loss: 6783.6256 - regularization_loss: 0.0000e+00 - total_loss: 6783.6256
Epoch 2/3
79/79 [==============================] - 6s 80ms/step - root_mean_squared_error: 1.0265 - factorized_top_k/top_1_categorical_accuracy: 4.5000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0156 - factorized_top_k/top_10_categorical_accuracy: 0.0367 - factorized_top_k/top_50_categorical_accuracy: 0.1724 - factorized_top_k/top_100_categorical_accuracy: 0.3003 - loss: 6460.8987 - regularization_loss: 0.0000e+00 - total_loss: 6460.8987
Epoch 3/3
79/79 [==============================] - 6s 79ms/step - root_mean_squared_error: 1.005

In [49]:
def predict_movie(user, top_n=3):
    # Create a model that takes in raw query features, and
    index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
    # recommends movies out of the entire movies dataset.
    index.index_from_dataset(
      tf.data.Dataset.zip((movies.batch(100), movies.batch(100).map(model.movie_model)))
    )

    # Get recommendations.
    _, titles = index(tf.constant([str(user)]))
    
    print('Top {} recommendations for user {}:\n'.format(top_n, user))
    for i, title in enumerate(titles[0, :top_n].numpy()):
        print('{}. {}'.format(i+1, title.decode("utf-8")))

In [55]:
predict_movie(933,top_n=50)

Top 50 recommendations for user 933:

1. Young Guns II (1990)
2. Cowboy Way, The (1994)
3. Real Genius (1985)
4. Escape from New York (1981)
5. Under Siege (1992)
6. Boomerang (1992)
7. Ace Ventura: Pet Detective (1994)
8. Cliffhanger (1993)
9. Coneheads (1993)
10. Son in Law (1993)


In [51]:
user_id=str(933)

# Filter the dataset for user_id = '123'
filtered_ratings = ratings.filter(lambda x: x['user_id'] == tf.constant(user_id))

# Iterate through the filtered dataset to access the records
for record in filtered_ratings:
    movie_title = record['movie_title'].numpy().decode('utf-8')
    user_rating = record['user_ratings'].numpy()
    print(f"Movie: {movie_title}, Rating: {user_rating}")

Movie: Birdcage, The (1996), Rating: 2.0
Movie: Braveheart (1995), Rating: 5.0
Movie: Return of the Pink Panther, The (1974), Rating: 2.0
Movie: Three Musketeers, The (1993), Rating: 3.0
Movie: Romeo Is Bleeding (1993), Rating: 3.0
Movie: Grease 2 (1982), Rating: 1.0
Movie: Natural Born Killers (1994), Rating: 1.0
Movie: Clockwork Orange, A (1971), Rating: 5.0
Movie: City Slickers II: The Legend of Curly's Gold (1994), Rating: 1.0
Movie: Star Wars (1977), Rating: 4.0
Movie: Sting, The (1973), Rating: 4.0
Movie: Fargo (1996), Rating: 5.0
Movie: Star Trek: The Wrath of Khan (1982), Rating: 4.0
Movie: Mulholland Falls (1996), Rating: 2.0
Movie: Star Trek IV: The Voyage Home (1986), Rating: 3.0
Movie: One Flew Over the Cuckoo's Nest (1975), Rating: 4.0
Movie: Aliens (1986), Rating: 3.0
Movie: Pulp Fiction (1994), Rating: 5.0
Movie: Tombstone (1993), Rating: 4.0
Movie: Silence of the Lambs, The (1991), Rating: 5.0
Movie: Under Siege 2: Dark Territory (1995), Rating: 1.0
Movie: Toy Story (19

In [39]:
import tensorflow as tf

user_id=str(933)

# Filter the dataset for user_id = '123'
filtered_ratings = ratings.filter(lambda x: x['user_id'] == tf.constant(user_id))

# Iterate through the filtered dataset to access the records
for record in filtered_ratings:
    movie_title = record['movie_title'].numpy().decode('utf-8')
    user_rating = record['user_ratings'].numpy()
    print(f"Movie: {movie_title}, Rating: {user_rating}")


Movie: Birdcage, The (1996), Rating: 2.0
Movie: Braveheart (1995), Rating: 5.0
Movie: Return of the Pink Panther, The (1974), Rating: 2.0
Movie: Three Musketeers, The (1993), Rating: 3.0
Movie: Romeo Is Bleeding (1993), Rating: 3.0
Movie: Grease 2 (1982), Rating: 1.0
Movie: Natural Born Killers (1994), Rating: 1.0
Movie: Clockwork Orange, A (1971), Rating: 5.0
Movie: City Slickers II: The Legend of Curly's Gold (1994), Rating: 1.0
Movie: Star Wars (1977), Rating: 4.0
Movie: Sting, The (1973), Rating: 4.0
Movie: Fargo (1996), Rating: 5.0
Movie: Star Trek: The Wrath of Khan (1982), Rating: 4.0
Movie: Mulholland Falls (1996), Rating: 2.0
Movie: Star Trek IV: The Voyage Home (1986), Rating: 3.0
Movie: One Flew Over the Cuckoo's Nest (1975), Rating: 4.0
Movie: Aliens (1986), Rating: 3.0
Movie: Pulp Fiction (1994), Rating: 5.0
Movie: Tombstone (1993), Rating: 4.0
Movie: Silence of the Lambs, The (1991), Rating: 5.0
Movie: Under Siege 2: Dark Territory (1995), Rating: 1.0
Movie: Toy Story (19

unique_user_ids

In [64]:
for record in filtered_ratings:
    test = record['movie_title'].numpy().decode('utf-8')

In [66]:
test

'Brazil (1985)'

In [ ]:
# Example of user interactions (movies and ratings)
user_movies = [movie_id_1, movie_id_2, movie_id_3]
user_ratings = [rating_1, rating_2, rating_3]

# Convert user interactions into tensors
user_movie_tensor = tf.convert_to_tensor(user_movies, dtype=tf.int32)
user_rating_tensor = tf.convert_to_tensor(user_ratings, dtype=tf.float32)

# Generate the user embedding using the fitted model
user_embedding = model.user_model({
    "movie_ids": user_movie_tensor,
    "ratings": user_rating_tensor
})

# Use the retrieval task to get top recommendations
recommendations, scores = model.retrieval_task.index({
    "user_embedding": user_embedding
}, k=10)

# Display the recommended movies and their scores
for title, score in zip(recommendations["movie_titles"], scores):
    print(f"Movie: {title}, Score: {score}")

# Data (in Pandas)

In [67]:
ratings_df = tfds.as_dataframe(ratings)

In [78]:
ratings_df.head(10)

,movie_title,user_id,user_ratings
0,"b""One Flew Over the Cuckoo's Nest (1975)""",b'138',4.0
1,b'Strictly Ballroom (1992)',b'92',2.0
2,"b'Very Brady Sequel, A (1996)'",b'301',4.0
3,b'Pulp Fiction (1994)',b'60',4.0
4,b'Scream 2 (1997)',b'197',3.0
5,b'Crash (1996)',b'601',4.0
6,b'Aladdin (1992)',b'710',3.0
7,b'True Romance (1993)',b'833',2.0
8,b'Bob Roberts (1992)',b'916',5.0
9,b'Starship Troopers (1997)',b'940',2.0


In [77]:
ratings_df[ratings_df['user_id'] == b'933']

,movie_title,user_id,user_ratings
262,"b'Birdcage, The (1996)'",b'933',2.0
1281,b'Braveheart (1995)',b'933',5.0
3158,"b'Return of the Pink Panther, The (1974)'",b'933',2.0
3852,"b'Three Musketeers, The (1993)'",b'933',3.0
4046,b'Romeo Is Bleeding (1993)',b'933',3.0
...,...,...,...
95007,b'Rosencrantz and Guildenstern Are Dead (1990)',b'933',3.0
95242,b'In the Name of the Father (1993)',b'933',4.0
95690,"b'Rock, The (1996)'",b'933',2.0
96808,"b'Man Without a Face, The (1993)'",b'933',3.0


In [79]:
movies_df = tfds.as_dataframe(movies)

In [80]:
movies_df.head(10)

,
0,b'You So Crazy (1994)'
1,b'Love Is All There Is (1996)'
2,b'Fly Away Home (1996)'
3,b'In the Line of Duty 2 (1987)'
4,"b'Niagara, Niagara (1997)'"
5,"b""Young Poisoner's Handbook, The (1995)"""
6,"b'Age of Innocence, The (1993)'"
7,b'Flirt (1995)'
8,b'Frisk (1995)'
9,b'unknown'
